In [12]:
import pyvisa as visa
import time
import numpy as np
import pymeasure
from pymeasure.instruments.keithley import Keithley2400
import pandas as pd
import matplotlib.pyplot as plt
import datetime
from labview_automation import LabVIEW
from MultiPyVu import MultiVuServer as mvs
from MultiPyVu import MultiVuClient as mvc

from IPython.display import clear_output

def nano_change_chanel(volt, chan=None):
    if not chan:
        return
    volt.write(":SENS:CHAN {}".format(chan))
    time.sleep(0.1)
    if volt.query(":SENS:CHAN?").strip() == str(chan):
        return
    else:
        print("channel switch error")
        raise

def init_hall_two_channel(dc_current=50e-6, step=3):
    for i in range(step):
        print("sample 1:" + str(i) + "chan1, std_1, chan_2,std_2")
        print(measure_resistance_two_channel(dc1, volt1, dc_current=dc_current))
    print("-"*10)
    for i in range(step):
        print("sample 2:" + str(i) + "chan1, std_1, chan_2,std_2")
        print(measure_resistance_two_channel(dc2, volt2, dc_current=dc_current))
        
def measure_resistance(dc, volt, dc_current=100e-6, measure_points=30, skip_points=10, time_per_point=0.1):
    dc.apply_current()  # Sets up to source current
    dc.source_current_range = 10e-3   # Sets the source current range to 10 mA
    dc.compliance_voltage = 10        # Sets the compliance voltage to 10 V
    dc.source_current = dc_current            # Sets the source current to 0 mA
    dc.enable_source() 
    v_up = []
    for i in range(30):  # apply positive current
        time.sleep(time_per_point)
        if i < skip_points:
            continue
        v_up.append(float(volt.query("fetch?")))
    v_up = sorted(v_up)
    v_up = v_up[3: -3]
    # Sets the compliance voltage to 10 V
    dc.source_current = -1 * dc_current
    dc.enable_source() 
    
    v_down = []
    for i in range(measure_points):   # apply negative current
        time.sleep(time_per_point)
        if i < skip_points:
            continue
        v_down.append(float(volt.query("fetch?")))
    
    dc.shutdown() 
    #dc.source_current = dc_current
    #dc.enable_source() 
    
    v_down = sorted(v_down)
    v_down = v_down[3: -3]
    
    average_v = (np.array(v_up).mean() - np.array(v_down).mean())/2
    std_v = (np.array(v_up).std() + np.array(v_down).std())/2
    return average_v, std_v


def measure_resistance_two_channel(dc, volt, dc_current=100e-6, measure_points=30, skip_points=10, time_per_point=0.05):
    #dc.apply_current()  # Sets up to source current
    dc.source_current_range = 10e-3   # Sets the source current range to 10 mA
    
    dc.compliance_voltage = 100        # Sets the compliance voltage to 100 V
    
    dc.source_current = dc_current            # Sets the source current to 0 mA
    dc.enable_source() 
    v_up_1 = []
    nano_change_chanel(volt, chan=1)
    for i in range(measure_points):  # apply positive current
        time.sleep(time_per_point)
        if i < skip_points:
            continue
        v_up_1.append(float(volt.query("fetch?")))
    v_up_1 = sorted(v_up_1)
    v_up_1 = v_up_1[3: -3]
    
    
    v_up_2 = []
    nano_change_chanel(volt, chan=2)
    for i in range(measure_points):  # apply positive current
        time.sleep(time_per_point)
        if i < skip_points:
            continue
        v_up_2.append(float(volt.query("fetch?")))
    v_up_2 = sorted(v_up_2)
    v_up_2 = v_up_2[3: -3]
    
    # Sets the compliance voltage to 10 V
    dc.source_current = -1 * dc_current
    dc.enable_source() 
    
    v_down_1 = []
    nano_change_chanel(volt, chan=1)
    for i in range(measure_points):   # apply negative current
        time.sleep(time_per_point)
        if i < skip_points:
            continue
        v_down_1.append(float(volt.query("fetch?")))
        
        
    v_down_2 = []
    nano_change_chanel(volt, chan=2)
    for i in range(measure_points):   # apply negative current
        time.sleep(time_per_point)
        if i < skip_points:
            continue
        v_down_2.append(float(volt.query("fetch?")))
    
    dc.shutdown() 

    
    v_down_1 = sorted(v_down_1)
    v_down_1 = v_down_1[3: -3]
    v_down_2 = sorted(v_down_2)
    v_down_2 = v_down_2[3: -3]
    
    average_v_1 = (np.array(v_up_1).mean() - np.array(v_down_1).mean())/2
    std_v_1 = (np.array(v_up_1).std() + np.array(v_down_1).std())/2
    
    average_v_2 = (np.array(v_up_2).mean() - np.array(v_down_2).mean())/2
    std_v_2 = (np.array(v_up_2).std() + np.array(v_down_2).std())/2
    return average_v_1, std_v_1, average_v_2, std_v_2

import subprocess
def get_position():
    result = subprocess.check_output([r'C:\Program Files\IronPython 2.7\ipy.exe', 
                                      r'C:\Users\yanglabuser\Desktop\Sam files\_switch\ppms-master\ppms_position.py'])

    current_p = round(float(result.decode('utf-8').strip().split(',')[1][:-1]))
    return current_p

def set_T(t=300):
    if t < 2 or t > 400:
        raise
    result = subprocess.check_output([r'C:\Program Files\IronPython 2.7\ipy.exe', 
                                      r'C:\Users\yanglabuser\Desktop\Sam files\_switch\ppms-master\ppms.py', str(t)])

    current_t = result.decode('utf-8').strip()
    if abs(float(current_t)-t)<1:
        print("current T {}".format(current_t))
        return
    else:
        print("set t:")
        print(t)
        print("current t:")
        print(current_t)
        raise
        
def set_Field(field=0, persistent=False):
    if abs(field) > 140000:
        raise
    if persistent:
        result = subprocess.check_output([r'C:\Program Files\IronPython 2.7\ipy.exe', 
                                      r'C:\Users\yanglabuser\Desktop\Sam files\_switch\ppms-master\ppms_set_field.py',
                                          str(field), str('persistent')])
    else:
        result = subprocess.check_output([r'C:\Program Files\IronPython 2.7\ipy.exe', 
                                          r'C:\Users\yanglabuser\Desktop\Sam files\_switch\ppms-master\ppms_set_field.py', 
                                          str(field)])

    current_field = result.decode('utf-8').strip()
    print(current_field)
    if abs(float(current_field)-field)<2:
        print("current field {}".format(current_field))
        return
    else:
        print("set field:")
        print(field)
        print("current t:")
        print(current_field)
        raise
        
def generate_fields(fileds, steps):
    assert (len(fileds) == len(steps))
    res = []
    for i in range(len(fileds)):
        res.extend([f for f in range(fileds[i], fileds[i + 1] if i+1 < len(fileds) else 0, 
                                   -1 * steps[i])])
    res = res + [0] + [-1* f for f in res[::-1]]
    res += res[::-1][1:]
    return res  

def field_scan_two_sample_two_channel(dc1, volt1, dc2, volt2, dc_current_1=1e-4, dc_current_2=1e-4, fields=[], Ts=[None], sample1_id="1", sample2_id="2",
                                      t_wait=10, field_wait=5):
    for T in Ts:
        if not T is None:
            set_T(T)
            print(datetime.datetime.now())
            time.sleep(t_wait*60)
        else:
            T = "300"
            
        file_1 = "data/" + sample1_id + "_" + str(T) + "K_" + str(int(dc_current_1*1e6)) + "uA"+ str(int(time.time())) + ".csv"
        file_2 = "/data/" + sample2_id + "_" + str(T) + "K_" + str(int(dc_current_2*1e6)) + "uA"+ str(int(time.time())) + ".csv"
        
        with open(file_1, 'w') as f:
            f.write("field(G), average_v_1, std_v_1, average_v_2, std_v_2, dc_current\n")
            
        with open(file_2, 'w') as f:
            f.write("field(G), average_v_1, std_v_1,  average_v_2, std_v_2, dc_current\n")
        dc_current = None
        for field in fields:
            set_Field(field)
            time.sleep(field_wait)
            for file in [file_1, file_2]:
                if file == file_1:
                    dc_current = dc_current_1
                    average_v_1, std_v_1, average_v_2, std_v_2 = measure_resistance_two_channel(dc=dc1, volt=volt1, dc_current=dc_current)
                else:
                    dc_current = dc_current_2
                    average_v_1, std_v_1, average_v_2, std_v_2 = measure_resistance_two_channel(dc=dc2, volt=volt2, dc_current=dc_current)
                    
                with open(file, 'a') as f:
                    f.write("{},{},{},{},{},{}\n".format(field, average_v_1, std_v_1, average_v_2, std_v_2, dc_current))

rm = visa.ResourceManager()
rm.list_resources()

('ASRL1::INSTR',
 'GPIB0::15::INSTR',
 'GPIB1::1::INSTR',
 'GPIB1::7::INSTR',
 'GPIB1::8::INSTR',
 'GPIB1::10::INSTR',
 'GPIB1::11::INSTR',
 'GPIB1::23::INSTR')

In [13]:
def SetField(Field=200, rate=100, Mode='persistent'):
    with mvs.MultiVuServer() as server:

    # Start the client
        with mvc.MultiVuClient() as client:
            if Mode=='persistent':
                client.set_field(Field,
                         rate, 
                         client.field.approach_mode.linear,
                         client.field.driven_mode.persistent)
            elif  Mode=='driven':
                client.set_field(Field,
                         rate, 
                         client.field.approach_mode.linear,
                         client.field.driven_mode.driven)
            else:
                print("wrong mode")

In [14]:
def GetField():
    with mvs.MultiVuServer() as server:

    # Start the client
        with mvc.MultiVuClient() as client:
    
            Field, sF = client.get_field()
        
    return Field, sF

In [15]:
def SetTemp(Temp=2, rate=10, Mode='fast_settle'):
    with mvs.MultiVuServer() as server:

    # Start the client
        with mvc.MultiVuClient() as client:
            if Mode=='fast_settle':
                client.set_temperature(Temp,
                         rate, 
                         client.temperature.approach_mode.fast_settle)
            elif  Mode=='no_overshoot':
                 client.set_temperature(Temp,
                         rate, 
                         client.temperature.approach_mode.no_overshoot)
            else:
                print("wrong mode")


In [16]:
def GetTemp():
    with mvs.MultiVuServer() as server:

    # Start the client
        with mvc.MultiVuClient() as client:
    
            Temp, sT = client.get_temperature()
        
    return Temp, sT

In [17]:
def SetTempStable(Temp=2, rate=10, Mode='fast_settle'):
    with mvs.MultiVuServer() as server:

    # Start the client
        with mvc.MultiVuClient() as client:
            if Mode=='fast_settle':
                client.set_temperature(Temp,
                         rate, 
                         client.temperature.approach_mode.fast_settle)
            elif  Mode=='no_overshoot':
                 client.set_temperature(Temp,
                         rate, 
                         client.temperature.approach_mode.no_overshoot)
            else:
                print("wrong mode")
            
            
            Flg = 1
            while  Flg:
                time.sleep(60)
                T = client.get_temperature()
                if T[1]=='Stable':
                    print('Temp is {}'.format(Temp))
                    Flg = 0
            


In [18]:
GetTemp()

MultiVuServer - Found PPMS running.
MultiVuServer - listening on ('localhost', 5000)
MultiVuServer - Press ESC to exit.
MultiVuClient - Starting connection to ('localhost', 5000)
MultiVuServer - Accepted connection from ('127.0.0.1', 59619)
MultiVuClient - Connected to PPMS MultiVuServer at ('127.0.0.1', 59619)
MultiVuServer - Client ('127.0.0.1', 59619) disconnected.
MultiVuClient - Closing connection to ('127.0.0.1', 59619)
MultiVuServer - Closing connection to ('127.0.0.1', 59619)


(2.0004, 'Stable')

In [19]:
SetField(Field=0, rate=100, Mode='persistent')

MultiVuServer - Found PPMS running.
MultiVuServer - listening on ('localhost', 5000)
MultiVuServer - Press ESC to exit.
MultiVuClient - Starting connection to ('localhost', 5000)
MultiVuServer - Accepted connection from ('127.0.0.1', 58920)
MultiVuClient - Connected to PPMS MultiVuServer at ('127.0.0.1', 58920)
MultiVuServer - Client ('127.0.0.1', 58920) disconnected.
MultiVuClient - Closing connection to ('127.0.0.1', 58920)
MultiVuServer - Closing connection to ('127.0.0.1', 58920)


In [20]:
volt_ID=1
volt = rm.open_resource('GPIB1::{}::INSTR'.format(volt_ID))

def LVread():
    lv = LabVIEW()
    lv.start() # Launches the active LabVIEW with the listener VI
    with lv.client() as c:
        control_values = {
        
            }
    
        indicators = c.run_vi_synchronous(r"D:\User Data\Cheng Yang\Sam files\_switch\test_labview_python\\PPMS_Status_Eckberg.vi", control_values)
    #  print(indicators['Temp (K)'],indicators['Angle'], indicators['Field'] )
    #lv.kill() # Stop LabVIEW
    T=float(indicators['Temp (K)'])
    Ang=float(indicators['Angle'])
    Field=float(indicators['Field'])
    return T, Ang, Field


def measure_voltage_two_channel(volt):

    nano_change_chanel(volt, chan=1)
    vxx=float(volt.query("fetch?"))

    nano_change_chanel(volt, chan=2)
    vxy =float(volt.query("fetch?"))
    

    return vxx, vxy, float(time.time())

def measure_voltage_one_channel(volt):

    nano_change_chanel(volt, chan=1)
    vxx=float(volt.query("fetch?"))
    vxy=0
    

    return vxx, vxy, float(time.time())


def measure_voltage_one_channel_average(volt):

    v_up = []
    for i in range(30):  # apply positive current
        time.sleep(0.1)
        if i < 10:
            continue
        v_up.append(float(volt.query("fetch?")))
    v_up = sorted(v_up)
    v_up = v_up[3: -3]
    # Sets the compliance voltage to 10 V
    
    vxx = np.array(v_up).mean()
    vxy=0    

    return vxx, vxy, float(time.time())

def WRITEvoltageinFile(volt, dc_current=0.1e-3, sample1_id="50nmWTe2",Temp='2K'):
        
#        Tmin, Tmax = Temp - 0.5, Temp + 0.5

        T = Temp
        Ang = -15    
        file = "data/" + sample1_id + "_" + str(T) + "K_" + str(int(dc_current*1e6)) + "uA"+ str(int(time.time())) + ".csv"
        
        with open(file, 'w') as f:
            f.write("t, Ang, Field, vxx, vxy, dc_current, T\n")
        
        vxy, vxx, t0 = measure_voltage_two_channel(volt)
        
        lv = LabVIEW()
        lv.start() # Launches the active LabVIEW with the listener VI
        with lv.client() as c:
            control_values = {
        
                        }
            
            t1st = None
        
            while int(t0) < 5637029254:
        
                Ang=Ang+5
                print(Ang)
                time.sleep(3)
                        
            
                indicators = c.run_vi_synchronous(r"D:\User Data\Cheng Yang\Sam files\_switch\test_labview_python\\PPMS_Status_Eckberg.vi", control_values)

                T=float(indicators['Temp (K)'])
#                Ang=float(indicators['Angle'])
                Field=float(indicators['Field'])
                vxy, vxx, t = measure_voltage_two_channel(volt)
                print(vxx,vxy)           
               # (a, b)=divmod(int(t-t0),5)
     #       if b == 0:
      #          T, Ang, Field= LVread() 
                
#                 if round(T)==100:
#                     Ang=Ang+20
#                 if round(T)==10:
#                     Ang=Ang+30    
                
#                if not t1st & (T >= Tmin) & (T <= Tmax):
#                    t1st = t

#                if t1st & ((T < Tmin) | (T > Tmax)):
#                    t1st = None
#                    continue
#                else:
#                   if t - t1st >= 1800:
                with open(file, 'a') as f:
                     f.write("{},{},{},{},{},{},{}\n".format(t-t0, Ang, Field, vxx, vxy, dc_current, T))
                        

def WRITEvoltageinFile_onechannel(volt, dc_current=0.1e-3, sample1_id="50nmWTe2"):

        T = "10"
        Ang = -15
        file = "data/" + sample1_id + "_" + str(T) + "K_" + str(int(dc_current*1e6)) + "uA"+ str(int(time.time())) + ".csv"
        
        with open(file, 'w') as f:
            f.write("t, Ang, Field, vxx, vxy, dc_current, T\n")
        
        vxx, vxy, t0 = measure_voltage_one_channel_average(volt)
        
        lv = LabVIEW()
        lv.start() # Launches the active LabVIEW with the listener VI
        with lv.client() as c:
            control_values = {
        
                        }
    
        
            while    int(t0) < 5637029254:
                
                Ang=Ang+5
                print(Ang)
                time.sleep(3)
                vxy, vxx, t = measure_voltage_one_channel_average(volt)
                print(vxx,vxy)

                        
            
                indicators = c.run_vi_synchronous(r"D:\User Data\Cheng Yang\Sam files\_switch\test_labview_python\\PPMS_Status_Eckberg.vi", control_values)

                T=float(indicators['Temp (K)'])
#               Ang=float(indicators['Angle'])
                Field=float(indicators['Field'])
                
               # (a, b)=divmod(int(t-t0),5)
     #       if b == 0:
      #          T, Ang, Field= LVread() 
                
                if round(T)==100:
                    Ang=Ang+20
                if round(T)==10:
                    Ang=Ang+30    
                    
                with open(file, 'a') as f:
                     f.write("{},{},{},{},{},{},{}\n".format(t-t0, Ang, Field, vxx, vxy, dc_current, T))

                        
def DC_field_measurement_onechannel(volt, FieldMax=10000, FieldMin=-10000, ramprate=100, input_I =0.014, sample_id="CoFeGd"):
    
    filename = "data/" + sample_id + "_{}.csv".format(int(time.time()))

    with open(filename, 'w') as f:
        f.write("Field,V,ac_I,Temp\n")
    # Start the server.
    with mvs.MultiVuServer() as server:

        # Start the client
        with mvc.MultiVuClient() as client:
        
            client.set_field(FieldMax, 100, 
                                     client.field.approach_mode.linear,
                                     client.field.driven_mode.persistent)

            Flg = 1
            while  Flg:
                F = client.get_field()
                if abs(F[0] - FieldMax)<1:
                    print('Field is {}'.format(FieldMax))
                    Flg = 0

            client.set_field(FieldMin, ramprate, 
                                     client.field.approach_mode.linear,
                                     client.field.driven_mode.persistent)

            Flg1 = 1
            while  Flg1:
                F = client.get_field()
                V = measure_voltage_one_channel(volt)
                T = [300]
                with open(filename, 'a') as f:
                          f.write("{},{},{},{}\n".format(F[0], V[0], input_I, T[0]))
                if abs(F[0] - FieldMin)<1:
                    print('Field is {}'.format(FieldMin))
                    Flg1 = 0

            client.set_field(FieldMax, ramprate, 
                                     client.field.approach_mode.linear,
                                     client.field.driven_mode.persistent)

            Flg2 = 1
            while  Flg2:
                F = client.get_field()
                V = measure_voltage_one_channel(volt)
                T = [300] 
                with open(filename, 'a') as f:
                          f.write("{},{},{},{}\n".format(F[0], V[0], input_I, T[0]))
                if abs(F[0] - FieldMax)<1:
                    print('Field is {}'.format(FieldMax))
                    Flg2 = 0            

            client.set_field(0, 100, 
                                client.field.approach_mode.linear,
                                client.field.driven_mode.persistent)

In [21]:
measure_voltage_one_channel(volt)

(-11.8928706, 0, 1668478235.5607016)

In [22]:
sensitivity = {1:26, 0.5:25, 0.2:24, 0.1:23, 0.05:22, 0.02:21, 0.01:20, 0.005:19, 0.002:18, 0.001:17, 0.0005:16, 0.0002:15,
               0.0001:14, 0.00005:13, 0.00002:12, 0.00001:11, 0.000005:10, 0.000002:9, 0.000001:8, 0.0000005:7, 0.0000002:6, 0.0000001:5, 0.00000005:4, 0.00000002:3,
               0.00000001:2, 0.000000005:1, 0.000000002:0}

timeconstant = {30000:19, 10000:18, 3000:17, 1000:16, 300:15, 100:14, 30:13, 10:12, 3:11, 1:10, 0.3:9, 0.1:8, 0.03:7,
                0.01:6, 0.003:5, 0.001:4, 0.0003:3, 0.0001:2, 0.00003:1, 0.00001:0}

class Lockin:
    def __init__(self, resource, gpib=1):
        self.lock_in = resource.open_resource('GPIB1::{}::INSTR'.format(gpib))

    def change_harmonic(self, harm=1, sens=1):
        self.lock_in.write("HARM {}".format(harm))
        self.lock_in.write("SENS {}".format(sensitivity[sens]))
    
    def change_timeconstant(self, tc=1):
        self.lock_in.write("OFLT {}".format(timeconstant[tc]))

    def lock_in_read(self):
        x = float(self.lock_in.query("OUTP? 1").strip())
        y = float(self.lock_in.query("OUTP? 2").strip())
        r = float(self.lock_in.query("OUTP? 3").strip())
        theta = float(self.lock_in.query("OUTP? 4").strip())
        return x, y, r, theta

    def lock_in_measure(self, count=10, time_step=0.1,
                        wait_before_measure=1,
                        harm=1, sens=0.00001):
#        self.change_harmonic(harm, sens)
        time.sleep(wait_before_measure)
        xs, ys, rs, thetas = [], [], [], []
        for i in range(count):
            x, y, r, theta = self.lock_in_read()
            xs += [x]
            ys += [y]
            rs += [r]
            thetas += [theta]
            time.sleep(time_step)
            
        xs = sorted(xs)
        ys=  sorted(ys)
        rs=  sorted(rs)
        thetas=sorted(thetas)
        
        if len(xs)>30:
            xs = xs[round(1/6*len(xs)): -round(1/6*len(xs))]
            ys = ys[round(1/6*len(ys)): -round(1/6*len(ys))]
            rs = rs[round(1/6*len(rs)): -round(1/6*len(rs))]
            thetas = thetas[round(1/6*len(thetas)): -round(1/6*len(thetas))]

        x_mean, x_std = np.mean(xs), np.std(xs)
        y_mean, y_std = np.mean(ys), np.std(ys)
        r_mean, r_std = np.mean(rs), np.std(rs)
        theta_mean, theta_std = np.mean(thetas), np.std(thetas)
        
        
        return x_mean, y_mean, r_mean, theta_mean, \
               x_std, y_std,r_std, theta_std


In [23]:
lock1 = Lockin(rm, 7)
lock2 = Lockin(rm, 8)
lock3 = Lockin(rm, 10)
lock4 = Lockin(rm, 11)

In [29]:
print(lock1.lock_in_measure(count=1, time_step=0.1, wait_before_measure=0.1))
print(lock2.lock_in_measure(count=1, time_step=0.1, wait_before_measure=0.1))
print(lock3.lock_in_measure(count=1, time_step=0.1, wait_before_measure=0.1))
print(lock4.lock_in_measure(count=1, time_step=0.1, wait_before_measure=0.1))

(0.0138817, -0.000840191, 0.0139075, -3.462, 0.0, 0.0, 0.0, 0.0)
(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0)
(-2.57047e-07, -1.22842e-06, 1.23773e-06, -101.34, 0.0, 0.0, 0.0, 0.0)
(-9.6, 4.84663e-06, -7.9349e-07, 4.90717e-06, 0.0, 0.0, 0.0, 0.0)


In [12]:
print(lock1.lock_in_measure(count=1, time_step=0.1, wait_before_measure=0.1))
print(lock3.lock_in_measure(count=1, time_step=0.1, wait_before_measure=0.1))

(166.854, -9.78713e-05, 0.000150443, 0.000179232, 0.0, 0.0, 0.0, 0.0)
(-2.3842e-07, -1.78815e-07, 2.98025e-07, -140.196, 0.0, 0.0, 0.0, 0.0)


In [ ]:
lock2.change_harmonic(harm=2, sens=5e-4)

In [ ]:
A=lock2.lock_in_measure(count=100, time_step=0.1, wait_before_measure=0.1)
print(A[1])

In [ ]:
lock2.change_harmonic(harm=1, sens=5e-3)

In [25]:
#A=lock1.lock_in_measure(count=10, time_step=0.1, wait_before_measure=0.1)
#B=lock2.lock_in_measure(count=10, time_step=0.1, wait_before_measure=0.1)
#C=lock3.lock_in_measure(count=10, time_step=0.1, wait_before_measure=0.1)
#D=lock4.lock_in_measure(count=10, time_step=0.1, wait_before_measure=0.1)
#lock1.change_harmonic(harm=1, sens=5e-3)
#lock2.change_harmonic(harm=1, sens=5e-3)
#lock1.change_timeconstant(tc=0.3)
#lock2.change_timeconstant(tc=0.3)
time.sleep(5)
A=lock1.lock_in_measure(count=10, time_step=0.1, wait_before_measure=0.1)
# print(A[0])
#B=lock2.lock_in_measure(count=1, time_step=0.1, wait_before_measure=0.1)
#print(B[0])
#lock1.change_harmonic(harm=2, sens=1e-4)
#lock1.change_timeconstant(tc=1)
#lock2.change_harmonic(harm=2, sens=5e-4)
#lock2.change_timeconstant(tc=1)
time.sleep(10)
#A=lock1.lock_in_measure(count=30, time_step=0.1, wait_before_measure=0.1)
#print(A[1])
B=lock3.lock_in_measure(count=100, time_step=0.1, wait_before_measure=0.1)
# print(B[1])
#lock1.change_harmonic(harm=1, sens=5e-3)
#lock2.change_harmonic(harm=1, sens=5e-3)
#lock1.change_timeconstant(tc=0.3)
#lock2.change_timeconstant(tc=0.3)

time.sleep(10)
C=lock4.lock_in_measure(count=100, time_step=0.1, wait_before_measure=0.1)
# print(C[0])
#D=lock4.lock_in_measure(count=100, time_step=0.1, wait_before_measure=0)
#print(D[2])

[ 5.00000000e+00  1.38817000e-02 -1.17103212e-06 -9.57881818e+00]


NameError: name 'results' is not defined

# Samuel's (kinda) automated measurement

In [ ]:
# Samuel's edit to automate data collection. The end result will be saved as a nx9 matrix 
# with columns being angle, lock1-x, lock1-y, lock2-x, lock-y, etc.

# LOOKS LIKE LOCKIN 4's X AND Y ARE FLIPPED?

# change this, wrapped in ''
save_location = 'D:\\User Data\\Hanshen Huang\\hanshen_11_14_2022.txt'

# measurement parameters
angle = 5
angle_increment = 5
counts = [10, 100, 100, 100]
time_steps = [0.1, 0.1, 0.1, 0.1]
wait_before_measures = [.1,.1,.1,.1]
sleep_timers = [5, 10, 10, 10]

# ------------------------------------

INIT_RESULTS = True

while angle <= 360:
#     input("Press enter to continue.")
    clear_output(wait=True)
    print("---------MEASURING-----------")
    
    print("Angle: ", angle)
    
    result = np.array([])
    result = np.append(result, angle) # add angle information
   
    lockins = [Lockin(rm, 7), Lockin(rm, 8), Lockin(rm, 10), Lockin(rm, 11)]
    
    time.sleep(5)
        
    for i in range (4):
        measure = lockins[i].lock_in_measure(count=counts[i], 
                                              time_step=time_steps[i], 
                                              wait_before_measure=wait_before_measures[i])
        result = np.append(result, np.array([measure[0], measure[1]]))
        time.sleep(sleep_timers[i])
        
    print("current result\n 6.2E", (result))
    
    if not INIT_RESULTS:
        print("past results\n 6.2E", (results))
    
    # remeasure if needed
    print("---------DONE-----------")
    print("\tremeasure current angle: r")
    print("\tmanually change angle: a")
    print("\tcontinue to next measurment: ENTER")
    
    action = input()
    if action=='r' or action=='R':
        continue
    elif action=='a' or action=='A'
        angle = input("next angle: ")
    else
        angle += angle_increment
        
    if INIT_RESULTS:
        results = result
        INIT_RESULTS = False
    else:
        results = np.vstack((results, result))

    #save results
    np.savetxt(save_location, results, delimiter=',')

Angle:  360
current result
 [ 3.60000000e+02  1.38817000e-02 -8.40191000e-04  0.00000000e+00
  0.00000000e+00 -2.60595470e-07 -1.20639379e-06 -9.56063636e+00
  4.82212106e-06]
past results
 [[ 5.00000000e+00  1.38817000e-02 -8.40191000e-04  0.00000000e+00
   0.00000000e+00 -2.39936924e-07 -1.19333470e-06 -9.66100000e+00
   4.78874848e-06]
 [ 1.00000000e+01  1.38817000e-02 -8.40191000e-04  0.00000000e+00
   0.00000000e+00 -2.22213288e-07 -1.16696864e-06 -9.55581818e+00
   4.76542318e-06]
 [ 1.50000000e+01  1.38817000e-02 -8.40191000e-04  0.00000000e+00
   0.00000000e+00 -2.29021955e-07 -1.21372500e-06 -9.86054545e+00
   4.84485348e-06]
 [ 2.00000000e+01 -2.18440000e-02  3.48493000e-03  0.00000000e+00
   0.00000000e+00  2.07534697e-06  1.05062015e-05  1.69309364e+02
  -1.71002045e-05]
 [ 2.50000000e+01  1.38789000e-02 -8.64033000e-04  0.00000000e+00
   0.00000000e+00 -2.43768106e-07 -1.16696091e-06 -1.06216364e+01
   4.82039955e-06]
 [ 3.00000000e+01  1.38789000e-02 -8.64033000e-04  0.00

In [ ]:
# time.sleep(1)
# A=lock1.lock_in_measure(count=1, time_step=0.1, wait_before_measure=0)
# print(A[0])
B=lock2.lock_in_measure(count=10, time_step=0.1, wait_before_measure=0)
print(B[1])


In [ ]:
Darray = np.zeros((1,120),dtype=float)   
for i in range(0,120):
    D=lock3.lock_in_measure(count=1, time_step=0, wait_before_measure=0.05)
    Darray[0,i] = D[1]
    
fig = plt.figure()
plt.plot(Darray[0, :])
print(np.mean(Darray))
print(np.std(Darray))

In [13]:
def lock_angular_measurement(lock1, lock2, lock3, lock4, first_range=0.01, second_range=0.00001, time_step=0.1,
                             input_I =0.014, wait_before_measure=30, 
                             lock_count=10, sample_id="Fe2O3/BST",  
                             time_per_point=1, time_out_count = 60*5):
    
    filename = "QS/" + sample_id + "_{}.csv".format(int(time.time()))
    Ang=-10
    with open(filename, 'w') as f:
        f.write("time,POS,x,y,r,theta,std_x,std_y,std_r,std_theta,x2,y2,r2,theta2,std_x2,std_y2,std_r2,std_theta2,x3,y3,r3,theta3,std_x3,std_y3,std_r3,std_theta3,x4,y4,r4,theta4,std_x4,std_y4,std_r4,std_theta4,ac_I,Field,Temp\n")
    
        
        
        t0 = float(time.time())
        
        lv = LabVIEW()
        lv.start() # Launches the active LabVIEW with the listener VI
        with lv.client() as c:
            control_values = {
        
                        }
    
        
            while    int(t0) < 5637029254:
                Ang=Ang+5
                print(Ang)
                indicators = c.run_vi_synchronous(r"D:\User Data\Cheng Yang\Sam files\_switch\test_labview_python\\PPMS_Status_Eckberg.vi", control_values)
                time.sleep(20)
                res1=lock1.lock_in_measure(count=3, time_step=0.1, wait_before_measure=0.1)
                res2=lock2.lock_in_measure(count=3, time_step=0.1, wait_before_measure=0.1)
                res4=lock4.lock_in_measure(count=20, time_step=0.1, wait_before_measure=0.1)
                res3=lock3.lock_in_measure(count=20, time_step=0.1, wait_before_measure=0.1)
                print('Vxx1=', res1[3])
                print('Vxy1=', res2[3])
                print('Vxx2=', res3[1])
                print('Vxy2=', res4[2])
                T=float(indicators['Temp (K)'])
#                Ang=float(indicators['Angle'])
                Field=float(indicators['Field'])
                if Field<0:
                    Ang=Ang+180
                t=float(time.time())
#                print("current position: {}".format(Ang))
#                print("current time: {}".format(t-t0))
                
#                if round(abs(Field))==50000:
                with open(filename, 'a') as f:
                      f.write("{},{},{},{},{},{},{},{},{},{},{},{},{},{},{},{},{},{},{},{},{},{},{},{},{},{},{},{},{},{},{},{},{},{},{},{},{}\n".format(t-t0, Ang, *res1, *res2, *res3, *res4, input_I, Field, T))
            
            
            
def lock_field_measurement(lock1, lock2, lock3, lock4, FieldMax=10000, FieldMin=-10000, ramprate=100, first_range=0.01, second_range=0.00001, time_step=0.1,
                             input_I =0.014, wait_before_measure=30, 
                             lock_count=10, sample_id="CoFeGd",  
                             time_per_point=1, time_out_count = 60*5, Temp=300):
    
    filename = "data/" + sample_id + "_" + str(Temp) + "K" + "_{}.csv".format(int(time.time()))

    with open(filename, 'w') as f:
        f.write("Field,x,y,r,theta,std_x,std_y,std_r,std_theta,x2,y2,r2,theta2,std_x2,std_y2,std_r2,std_theta2,x3,y3,r3,theta3,std_x3,std_y3,std_r3,std_theta3,x4,y4,r4,theta4,std_x4,std_y4,std_r4,std_theta4,ac_I,Temp\n")
    # Start the server.
    with mvs.MultiVuServer() as server:

        # Start the client
        with mvc.MultiVuClient() as client:
        
            client.set_field(FieldMax, 100, 
                                     client.field.approach_mode.linear,
                                     client.field.driven_mode.persistent)

            Flg = 1
            while  Flg:
                F = client.get_field()
                if abs(F[0] - FieldMax)<1:
                    print('Field is {}'.format(FieldMax))
                    Flg = 0

            client.set_field(FieldMin, ramprate, 
                                     client.field.approach_mode.linear,
                                     client.field.driven_mode.persistent)

            Flg1 = 1
            while  Flg1:
                res1=lock1.lock_in_measure(count=1, time_step=0.1, wait_before_measure=0)
                res2=lock2.lock_in_measure(count=1, time_step=0.1, wait_before_measure=0)
                res3=res1
                res4=res2
                F = client.get_field()
                T = [Temp]
                with open(filename, 'a') as f:
                          f.write("{},{},{},{},{},{},{},{},{},{},{},{},{},{},{},{},{},{},{},{},{},{},{},{},{},{},{},{},{},{},{},{},{},{},{}\n".format(F[0], *res1, *res2, *res3, *res4, input_I, T[0]))
                if abs(F[0] - FieldMin)<1:
                    print('Field is {}'.format(FieldMin))
                    Flg1 = 0

            client.set_field(FieldMax, ramprate, 
                                     client.field.approach_mode.linear,
                                     client.field.driven_mode.persistent)

            Flg2 = 1
            while  Flg2:
                res1=lock1.lock_in_measure(count=1, time_step=0.1, wait_before_measure=0.1)
                res2=lock2.lock_in_measure(count=1, time_step=0.1, wait_before_measure=0.1)
                res3=res1
                res4=res2
                F = client.get_field()
                T = [Temp] 
                with open(filename, 'a') as f:
                          f.write("{},{},{},{},{},{},{},{},{},{},{},{},{},{},{},{},{},{},{},{},{},{},{},{},{},{},{},{},{},{},{},{},{},{},{}\n".format(F[0], *res1, *res2, *res3, *res4, input_I, T[0]))
                if abs(F[0] - FieldMax)<1:
                    print('Field is {}'.format(FieldMax))
                    Flg2 = 0            

            client.set_field(0, 100, 
                                client.field.approach_mode.linear,
                                client.field.driven_mode.persistent)
            
            
            
    
            
def lock_stepfield_measurement(lock1, lock2, lock3, lock4, Field=[100],  first_range=0.01, second_range=0.00001, time_step=0.1,
                             input_I =0.014, wait_before_measure=30, 
                             lock_count=10, sample_id="CoFeGd",  
                             time_per_point=1, time_out_count = 60*5, Temp=300):      
    filename = "data/" + sample_id + "_" + str(Temp) + "K" + "_{}.csv".format(int(time.time()))

    with open(filename, 'w') as f:
        f.write("Field,x,y,r,theta,std_x,std_y,std_r,std_theta,x2,y2,r2,theta2,std_x2,std_y2,std_r2,std_theta2,x3,y3,r3,theta3,std_x3,std_y3,std_r3,std_theta3,x4,y4,r4,theta4,std_x4,std_y4,std_r4,std_theta4,ac_I,Temp\n")
    # Start the server.
    with mvs.MultiVuServer() as server:

        # Start the client
        with mvc.MultiVuClient() as client:
            
            for f in Field:
        
                client.set_field(f, 100, 
                                         client.field.approach_mode.linear,
                                         client.field.driven_mode.driven)

                Flg = 1
                while  Flg:
                    F = client.get_field()
                    if abs(F[0] - f)<1:
                        print('Field is {}'.format(f))
                        Flg = 0


                res1=lock1.lock_in_measure(count=1, time_step=0.1, wait_before_measure=0)
                res2=lock2.lock_in_measure(count=10, time_step=0.1, wait_before_measure=0)
                res3=res1
                res4=res2
                T = [Temp]
                with open(filename, 'a') as f:
                          f.write("{},{},{},{},{},{},{},{},{},{},{},{},{},{},{},{},{},{},{},{},{},{},{},{},{},{},{},{},{},{},{},{},{},{},{}\n".format(F[0], *res1, *res2, *res3, *res4, input_I, T[0]))


            

            client.set_field(0, 100, 
                                client.field.approach_mode.linear,
                                client.field.driven_mode.persistent)
            

In [11]:
SetTempStable(Temp=300, rate=10)

--- Logging error ---
Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\lib\logging\handlers.py", line 70, in emit
    self.doRollover()
  File "C:\ProgramData\Anaconda3\lib\logging\handlers.py", line 171, in doRollover
    self.rotate(self.baseFilename, dfn)
  File "C:\ProgramData\Anaconda3\lib\logging\handlers.py", line 111, in rotate
    os.rename(source, dest)
PermissionError: [WinError 32] The process cannot access the file because it is being used by another process: 'D:\\User Data\\Cheng Yang\\Sam files\\_switch\\QdMultiVu.log' -> 'D:\\User Data\\Cheng Yang\\Sam files\\_switch\\QdMultiVu.log.1'
Call stack:
  File "C:\ProgramData\Anaconda3\lib\runpy.py", line 194, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "C:\ProgramData\Anaconda3\lib\runpy.py", line 87, in _run_code
    exec(code, run_globals)
  File "C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "C

Temp is 300


MultiVuServer - Closing connection to ('127.0.0.1', 58125)
--- Logging error ---
Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\lib\logging\handlers.py", line 70, in emit
    self.doRollover()
  File "C:\ProgramData\Anaconda3\lib\logging\handlers.py", line 171, in doRollover
    self.rotate(self.baseFilename, dfn)
  File "C:\ProgramData\Anaconda3\lib\logging\handlers.py", line 111, in rotate
    os.rename(source, dest)
PermissionError: [WinError 32] The process cannot access the file because it is being used by another process: 'D:\\User Data\\Cheng Yang\\Sam files\\_switch\\QdMultiVu.log' -> 'D:\\User Data\\Cheng Yang\\Sam files\\_switch\\QdMultiVu.log.1'
Call stack:
  File "C:\ProgramData\Anaconda3\lib\runpy.py", line 194, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "C:\ProgramData\Anaconda3\lib\runpy.py", line 87, in _run_code
    exec(code, run_globals)
  File "C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py", l

In [ ]:
lock_field_measurement(lock1, lock2, lock3, lock4, FieldMax=50, FieldMin=-50, ramprate=100, input_I =0.1, sample_id="CoFeGd", Temp=300)


In [ ]:
Ts=[300,200,100]
for T in Ts:
    SetTempStable(Temp=T, rate=10)
    time.sleep(120)
    lock_field_measurement(lock1, lock2, lock3, lock4, FieldMax=50, FieldMin=-50, ramprate=100, input_I =0.1, sample_id="Mn3Sn", Temp=T)


In [ ]:
fs = [400]
steps = [10]
fields = generate_fields(fs, steps)
print(fields)
lock_stepfield_measurement(lock1, lock2, lock3, lock4, Field=fields, input_I =0.1, sample_id="CoFeGd_OOP_3269_3270", Temp=300)
# for i in range(199):
#         time.sleep(2)
#         ac_1.write("SOUR:WAVE:AMPL {}".format((-i+200)*1e-5))                   
#         print ((-i+200)*1e-2,'mA')
# ac_1.write("SOUR:WAVE:ABOR")
# ac_1.write("SOUR:WAVE:RANG FIXED")
# SetTempStable(Temp=300, rate=10)



In [ ]:
DC_field_measurement_onechannel(volt, FieldMax=100, FieldMin=-100, ramprate=10, input_I =0.1, sample_id="CoFeGd_DC_0deg")

In [ ]:
lock_angular_measurement(lock1, lock2, lock3, lock4, input_I =0.014, sample_id="Ta_CoFeB_300K_Rxx_Rxy_Rxx2_Rxy2_2T_300K_0p45mA")

In [ ]:
LVread()[0]

In [ ]:
WRITEvoltageinFile_onechannel(volt, dc_current=0.1e-3, sample1_id="5nmPt_20nmFGT_100K_oop")

In [ ]:
t=1
while    int(t) < 5637029254:
    time.sleep(1)
    if round(LVread()[0])==2:
        WRITEvoltageinFile(volt, dc_current=1e-4, sample1_id="BST_Fe2O3_9T_0p01mA",Temp='2')

In [ ]:
measure_voltage_two_channel(volt)

In [ ]:
measure_voltage_one_channel_average(volt)

In [ ]:
def setcurr(curr=1, count_write=50, thrd=0.01):    
    for i in range(count_write):
            magnet.write("CURR {}".format(curr))
            time.sleep(0.1)
    print ('Set current done.', curr)

def covert_to_float(s):
    ans = ''
    for i in range(len(s) - 2):
        ans += s[i]
        if s[i+1].isalpha() & s[i+2].isalpha(): break
    return float(ans)

def fieldmeasure():
    s = gauss.query(":FETCH?")
    field=covert_to_float(s)*10000
    return field

In [ ]:
def field_scan_one_sample_two_channel(dc1, volt1, dc_current_1=1e-4, fields=[], Ts=[None], sample1_id="1", sample2_id="2",
                                      t_wait=10, field_wait=5):
    for T in Ts:
        if not T is None:
            set_T(T)
            print(datetime.datetime.now())
            time.sleep(t_wait*60)
        else:
            T = "300"
            
        file_1 = "data/" + sample1_id + "_" + str(T) + "K_" + str(int(dc_current_1*1e6)) + "uA"+ str(int(time.time())) + ".csv"
        
        with open(file_1, 'w') as f:
            f.write("field(G), average_v_1, std_v_1, average_v_2, std_v_2, dc_current\n")
            
        
        dc_current = None
        for field in fields:
            set_Field(field)
            time.sleep(field_wait)
            for file in [file_1]:
                if file == file_1:
                    dc_current = dc_current_1
                    average_v_1, std_v_1, average_v_2, std_v_2 = measure_resistance_two_channel(dc=dc1, volt=volt1, dc_current=dc_current)
                   
                with open(file, 'a') as f:
                    f.write("{},{},{},{},{},{}\n".format(field, average_v_1, std_v_1, average_v_2, std_v_2, dc_current))

In [ ]:
set_Field(0)

In [ ]:
#hall measurement connection, chan 1 rxy, chan 2 rxx
dc1_ID=30
dc2_ID=30
volt1_ID=8
volt2_ID=8
dc1 = Keithley2400("GPIB::{}".format(dc1_ID))
dc2 = None #Keithley2400("GPIB::{}".format(dc2_ID))
volt1 = rm.open_resource('GPIB0::{}::INSTR'.format(volt1_ID))
volt2 = None # rm.open_resource('GPIB0::{}::INSTR'.format(volt2_ID))

#volt2.write("*RST")
#volt1.write("*RST")

In [ ]:
volt1.write("*RST")

In [ ]:
init_hall_two_channel(step=2)

In [ ]:
fs = [140000]
steps = [5000]
fields = generate_fields(fs, steps)
print(fields)
temperatures = [355, 325, 300]
sample1_id='Xd806_1p5nmPt3nmnmTa1nmPt'
sample2_id='2'

In [ ]:
field_scan_two_sample_two_channel(dc1, volt1, dc2, volt2, dc_current_1=1e-4, dc_current_2=1e-4, 
                                  fields=fields, Ts=temperatures, sample1_id=sample1_id, sample2_id=sample2_id,
                                      t_wait=10, field_wait=5)
#set_Field(field=0, persistent=True)

In [ ]:
field_scan_one_sample_two_channel(dc1, volt1, dc_current_1=5e-5, fields=fields, Ts=temperatures, sample1_id=sample1_id, sample2_id="2",
                                      t_wait=10, field_wait=5)


set_Field(0, True)

In [ ]:
set_Field(0, True)

In [ ]:
set_T(300)

In [ ]:
! pip install MultiPyVu

In [ ]:
from MultiPyVu import MultiVuServer as mvs
from MultiPyVu import MultiVuClient as mvc

In [ ]:
with mvs.MultiVuServer() as server:

    # Start the client
    with mvc.MultiVuClient() as client:
        
        time.sleep(5)
        
        # A basic loop that demonstrates communication between 
        # client/server
        client.set_field(200.0,
                         100, 
                         client.field.approach_mode.linear,
                         client.field.driven_mode.persistent)

In [ ]:
# -*- coding: utf-8 -*-
'''
example_Local.py - This example tests local operation on the MultiVu PC.
'''

# Package used to insert a short wait time between data polls
import time

# Both the client and server must be running on the same machine 
# for 'local' operation.
from MultiPyVu import MultiVuClient as mvc
from MultiPyVu import MultiVuServer as mvs


# Start the server.
with mvs.MultiVuServer() as server:

    # Start the client
    with mvc.MultiVuClient() as client:
        
        time.sleep(5)
        
        # A basic loop that demonstrates communication between 
        # client/server
        client.set_field(200.0,
                         100, 
                         client.field.approach_mode.linear,
                         client.field.driven_mode.persistent)
        # Wait for 20 seconds after temperature and field are stable
        print('Waiting ')
        client.wait_for(10,
                        timeout_sec=0)

        for t in range(5):
            # Polls MultiVu for the temperature, field, and their
            #  respective states
            T, sT = client.get_temperature()
            F, sF = client.get_field()

            # Relay the information from MultiVu
            message = f'The temperature is {T}, status is {sT}; '
            message += f'the field is {F}, status is {sF}. '
            print(message)

            # collect data at roughly 2s intervals
            time.sleep(2)

In [ ]:
signalgeneratorID=21
microwave = rm.open_resource('GPIB1::{}::INSTR'.format(signalgeneratorID))

In [ ]:
# -*- coding: utf-8 -*-
'''
example_Local.py - This example tests local operation on the MultiVu PC.
'''

# Package used to insert a short wait time between data polls
import time

# Both the client and server must be running on the same machine 
# for 'local' operation.
from MultiPyVu import MultiVuClient as mvc
from MultiPyVu import MultiVuServer as mvs


# Start the server.
with mvs.MultiVuServer() as server:

    # Start the client
    with mvc.MultiVuClient() as client:
        
        time.sleep(5)
        
        # A basic loop that demonstrates communication between 
        # client/server
        client.set_field(10000.0,
                         100, 
                         client.field.approach_mode.linear,
                         client.field.driven_mode.persistent)
        F, sF = client.get_field()
        Flg = 1
        while  Flg:
            F = client.get_field()
            if F[0] > 10000.0:
                Flg = 0
        
        # Wait for 15 seconds
        print('Waiting')
        client.wait_for(15,
                        timeout_sec=0)
        client.set_field(0.0,
                         100, 
                         client.field.approach_mode.linear,
                         client.field.driven_mode.persistent)
        
        Flg = 1
        while Flg:
            F = client.get_field()
            if F[0] < 5500.0:
                microwave.write(":POW:STAT ON;")
                microwave.write(":POW:MODE FIX;LEV {};".format(0))
                microwave.write(":FREQ:CW {} Hz;:FREQ:MODE FIX;".format(12e9))
                Flg = 0
        
        Flg = 1
        while Flg:
            F1 = client.get_field()
            B=lock2.lock_in_measure(count=1, time_step=0.1, wait_before_measure=0.1)
            F2 = client.get_field()
            F = (F1[0] + F2[0])/2
            
            print(F, B[0])
            
            if F2[0] < 200.0:
                microwave.write(":POW:STAT OFF;")
                Flg = 0

                
        for t in range(5):
            # Polls MultiVu for the temperature, field, and their
            #  respective states
            T, sT = client.get_temperature()
            F, sF = client.get_field()

            # Relay the information from MultiVu
            message = f'The temperature is {T}, status is {sT}; '
            message += f'the field is {F}, status is {sF}. '
            print(message)

            # collect data at roughly 2s intervals
            time.sleep(2)

In [ ]:
! py --version


In [ ]:
! py run_server.py

In [ ]:
pwd

In [ ]:
cd C:\ProgramData\Anaconda3\Lib\site-packages\MultiPyVu

In [ ]:
! py CommandMultiVu.py

In [ ]:
signalgeneratorID=21
microwave = rm.open_resource('GPIB1::{}::INSTR'.format(signalgeneratorID))
microwave.write(":POW:STAT ON;")
microwave.write(":POW:STAT OFF;")
microwave.write(":POW:MODE FIX;LEV {};".format(power))
microwave.write(":FREQ:CW {} Hz;:FREQ:MODE FIX;".format(signalgeneratorfre))

In [ ]:
with mvs.MultiVuServer() as server:

    # Start the client
    with mvc.MultiVuClient() as client:
        client.set_field(0.0,
                                 50, 
                                 client.field.approach_mode.linear,
                                 client.field.driven_mode.persistent)
        for i in range(1000):
            F1 = client.get_field()
            A=lock1.lock_in_measure(count=1, time_step=0.1, wait_before_measure=0)
            F2 = client.get_field()
            print('Field 1 = ',F1[0], 'Field 2 = ', F2[0], '\n')